In [1]:
import pandas as pd
import numpy as np
from keras.models import load_model

import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


/Users/pavithraraghavan/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv('/Users/pavithraraghavan/Documents/Sentiment_Analysis/Final_datasets/Dataset_6.csv', encoding="latin1")
data.head(5)
#0 is sentiment negative
#1 is sentiment positive

,Sentiment,SentimentText
0,2,any idea when flight 703 to Miami will leave ...
1,1,"@AIIisonIraheta awesome, thanks ill be there"
2,1,You definitely will!
3,0,"In fact, this stinker smells like a direct-to-..."
4,0,*sigh* it's taking so long


In [3]:
print(data[ data['Sentiment'] == 0].size)

110166


In [4]:
print(data[ data['Sentiment'] == 1].size)

120946


In [5]:
print(data[ data['Sentiment'] == 2].size)

6198


In [6]:
pd.get_dummies(data['Sentiment']).values

array([[0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0]], dtype=uint8)

In [7]:
#data transformation
data['SentimentText'] = data['SentimentText'].apply(lambda x: x.lower())
data['SentimentText'] = data['SentimentText'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [8]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['SentimentText'].values)
X = tokenizer.texts_to_sequences(data['SentimentText'].values)
X = pad_sequences(X)

In [9]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 9837, 128)         256000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
#train, test
Y = pd.get_dummies(data['Sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(79498, 9837) (79498, 3)
(39157, 9837) (39157, 3)


In [11]:
model1 = load_model('/Users/pavithraraghavan/Documents/Sentiment_Analysis/saved_models/mymodel_3.h5')

/anaconda3/lib/python3.6/site-packages/keras/models.py:291: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [12]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')

In [13]:
np.save('/Users/pavithraraghavan/Documents/Sentiment_Analysis/val_test/xtrain', X_train)

In [14]:
np.save('/Users/pavithraraghavan/Documents/Sentiment_Analysis/val_test/ytrain', Y_train)

In [15]:
x_test = np.load('/Users/pavithraraghavan/Documents/Sentiment_Analysis/val_test/xtest.npy')

In [16]:
y_test = np.load('/Users/pavithraraghavan/Documents/Sentiment_Analysis/val_test/ytest.npy')

In [17]:
validation_size = 15000
X_validate = x_test[-validation_size:]
Y_validate = y_test[-validation_size:]
X_test = x_test[:-validation_size]
Y_test = y_test[:-validation_size]
score,acc = model1.evaluate(X_test, Y_test, verbose = 1, batch_size = 32)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))


  160/24157 [..............................] - ETA: 59:28

KeyboardInterrupt: 

In [50]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['SentimentText'].values)

In [55]:
input = "This is in Boston"

In [56]:
input = tokenizer.texts_to_sequences(input)
input = pad_sequences(input, maxlen=9837, dtype='int32', value=0)
sentiment = model1.predict(input,batch_size=1)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")
else:
    print("neutral")

positive
